# Learning a quadratic pseudo-metric from distance measurements

Recall that pseudo-metric is a generalization of a metric space in which the distance between two distinct points can be zero.
We are given a set of $N$ pairs of points in $\mathbf{R}^n$, $x_1, \ldots, x_N$, and $y_1, \ldots, y_N$, together with a set of distances $d_1, \ldots, d_N > 0$.
  The goal is to find (or estimate or learn) a quadratic pseudo-metric $d$
  $$d(x,y) =  \left( (x-y)^T P(x-y) \right)^{1/2},$$
  $P\in \mathbf{S}^n_{+}$, which approximates the given distances, i.e., $d(x_i, y_i) \approx d_i$. (The pseudo-metric $d$ is a metric only when $P \succ 0$; when $P\succeq 0$ is singular, it is a pseudo-metric.)
  
  To do this, we will choose $P\in \mathbf{S}^n_+$ that minimizes the mean squared error objective
  
  $$f(S)=\frac{1}{N}\sum_{i=1}^N (d_i - d(x_i,y_i))^2.$$
  
  ### Theoretical part.
  1. Show that the objective function $f$ is convex (Hint: expand the square and see what happens.)
  2. Show that the convex program $\text{minimize }f(S)$, $S\succeq 0$ can be expressed by an equivalent conic program with linear objective and a number of conic constraints using the $R^n_+$ (nonnegative orthant cone), $Q^n$ (second order cone), $Q_r^n$ (rotated second order cone), $S^n_+$ (positive semidefinite cone).
  
  ### Programming Part
  1. Solve the program $\text{minimize }f(S)$, $S\succeq 0$, preferably using a modelling package like ``cvxpy``. Note that "under the hood" your modelling package translates the program to the conic form in point 2. above.
  2. Use the obtained $P$ to measure the mean square error for the test data ``X_test``, ``Y_test``, ``d_test``.
  
---- 
*This exercise originates from "Additional Exercises" collection for Convex Optimization textbook of S. Boyd and L. Vandenberghe. Used under permission*

## Solution
--------

### Theoretical Part

##### 1) Show that the objective function $f$ is convex - Note: The exersize says $f(S)$ but since our variable is P, I think it makes sense to refer to it as $f(P)$ instead

To show that the objective function $\( f(P) \)$ is convex, we first expand the expression for $\( d(x_i, y_i) \)$:
$d(x_i, y_i) = (x_i - y_i)^T P (x_i - y_i) ^{1/2}.$

Substitute this into the objective function $\( f(P) \)$:
$f(P) = \frac{1}{N} \sum_{i=1}^N ( d_i - \left( (x_i - y_i)^T P (x_i - y_i) \right)^{1/2})^2.$

Let $\( z_i = x_i - y_i \)$. Then, the objective function becomes:
$f(P) = \frac{1}{N} \sum_{i=1}^N (d_i - \left( z_i^T P z_i \right)^{1/2})^2.$

To prove convexity, we need to show that $\( f(P) \)$ is a convex function of $\( P \)$. By expanding the square we end up:
$f(P) = \frac{1}{N} \sum_{i=1}^N ( d_i^2 - 2 d_i (z_i^T P z_i)^{1/2} + z_i^T P z_i).$

We can ignore the $\frac{1}{N}$ for our analysis, end we end up with a sum of terms that look like:

$d_i^2 - 2 d_i (z_i^T P z_i)^{1/2} + z_i^T P z_i$

For each term:
- $\( d_i^2 \)$ is constant with respect to \( P \).
- $\( z_i^T P z_i \)$ is linear in $\( P \)$ because $\( P \in \mathbf{S}^n_+ \)$, and the product $\( z_i^T P z_i \)$ is a linear function of $\( P \)$.
- The square root of the affine function $z_i^T P z_i$ is concave and multiplying by a negative constant $-2 d_i$ makes it convex. $-2 d_i (z_i^T P z_i)^{1/2}$ is convex.

Therefore, each term of the summation is a sum of a constant, a linear term, and a convex term, thus a convex term. The sum of convex terms is convex, which makes $\( f(P) \)$ a convex function.



### Theoretical Part

##### 2) Show that the convex program can be expressed by an equivalent conic program with linear objective and a number of conic constraints using the $R^n_+$ (nonnegative orthant cone), $Q^n$ (second order cone), $Q_r^n$ (rotated second order cone), $S^n_+$ (positive semidefinite cone).

#### Conversion to conic problem with linear objective

We can start by defining  $t_i$ to represent $\sqrt{((x_i - y_i)^T P (x_i - y_i))}$:

- $t_i = \sqrt{(x_i - y_i)^T P (x_i - y_i)}$

Then we can rewrite the objective function using slack variables $t_i$ and $u_i$ to represent $\((d_i - t_i)^2\)$:
- $u_i = (d_i - t_i)^2 $

Our goal is to minimize:
$f(u) = \frac{1}{N} \sum_{i=1}^N u_i.$ or the equivalent $f(u) = \sum_{i=1}^N u_i.$ or even $f(u) = \quad 1^T \mathbf{u}.$ with $\mathbf{u}$ a N-dimensional vector with N the size of the datapoints.

#### Constraints

1. **Second-Order Cone (SOC) Constraint for $t_i$**:
   $$ t_i^2 \geq (x_i - y_i)^T P (x_i - y_i), \quad \forall i $$
   $$
   \| \begin{pmatrix} t_i \\ (x_i - y_i)^T P^{1/2} \end{pmatrix}\|_2 \leq t_i
   $$
   $$
   (t_i, (x_i - y_i)^T P (x_i - y_i)) \in Q^n
   $$
   where
   $$
   Q^n = \{(t, x) \mid \|x\|_2^2 \leq t\}
   $$


2. **Rotated Second-Order Cone (RSOC) Constraint for $u_i$**:
   We can express $\( (d_i - t_i)^2 \leq u_i \)$:
   $$
   \| \begin{pmatrix} 2\sqrt{u_i} \\ d_i - t_i \end{pmatrix} \|_2 \leq d_i + t_i, \quad \forall i
   $$

   $$
   (u_i, 1, d_i - t_i) \in Q_r^n
   $$
   where
   $$
   Q_r^n = \{(u, v, w) \mid 2uv \geq w^2, u \geq 0, v \geq 0 \}
   $$

3. **Positive Semidefinite Cone (PSD) Constraint**:
   $$ P \succeq 0 $$ or $$P \in S^n_+$$

4. **Non-Negativity Constraint**:
   $$ u_i \geq 0, \quad t_i \geq 0, \quad \forall i $$ or $$u_i, t_i \in R^n_+$$

The final conic program is:

$$
\text{minimize} \quad 1^T \mathbf{u}
$$

subject to:

$$
| \begin{pmatrix} t_i \\ (x_i - y_i)^T P^{1/2} \end{pmatrix} \|_2 \leq t_i, \quad \forall i \quad \text{(SOC)}
$$

$$
\| \begin{pmatrix} 2\sqrt{u_i} \\ d_i - t_i \end{pmatrix} \|_2 \leq d_i + t_i, \quad \forall i \quad \text{(RSOC)}
$$

$$
P \in S^n_+
$$

$$
u_i \in R^n_+, t_i \in R^n_+, \forall i \quad \text{(Non-Negativity)}
$$





In [12]:
import cvxpy as cp
import numpy as np
from scipy import linalg as la

In [66]:
# In this box we generate the input data

np.random.seed(5680)

n = 5 # Dimension
N = 100 # Number of samples

P = np.random.randn(n,n)
P = P.dot(P.T) + np.identity(n)
sqrtP = la.sqrtm(P)

x = np.random.randn(N,n)
y = np.random.randn(N,n)

d = np.linalg.norm(sqrtP.dot((x-y).T),axis=0)    # distances according to metric P
d = np.maximum(d+np.random.randn(N),0)           # add random noise

N_test = 10 # Samples for test set
X_test = np.random.randn(N_test,n)
Y_test = np.random.randn(N_test,n)
d_test = np.linalg.norm(sqrtP.dot((X_test-Y_test).T),axis=0)  # distances according to metric P
d_test = np.maximum(d_test+np.random.randn(N_test),0)         # add random noise


#### My solution

In [68]:
### Training ###
P_approx = cp.Variable((n, n), PSD=True) # Let's setup our approximation of P matrix
objective = 0 # Let's initialize our objective
# Adding all terms of the form d_i**2
for distance in d:
    objective += distance**2
# Adding all terms of the form - 2d_i (x_i - y_i)^T P (x_i - y_i))^{1/2}
for distance, x_i, y_i in zip(d, x, y):
    objective += -2*distance*cp.sqrt(cp.quad_form((x_i - y_i),P_approx))
# Adding all terms of the form  (x_i - y_i)^T P (x_i - y_i))
for x_i, y_i in zip(x, y):
    objective += cp.quad_form((x_i - y_i),P_approx)

problem = cp.Problem(cp.Minimize(objective / N))
tre = problem.solve()
print(f"Training Error is {tre}")

Training Error is 0.720322118721639


In [70]:
### Testing ###
d_guessed = np.linalg.norm(la.sqrtm(P_approx.value).dot((X_test-Y_test).T),axis=0)
tse = (np.linalg.norm(d_test - d_guessed)**2)/N_test
print(f"Test Error is {tse}")


Test Error is 0.9157737162514736
